In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import random
import os, sys
import timm
import torch.nn.functional as F
from glob import glob
from PIL import Image
import cv2
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import albumentations as A
from torch.optim import AdamW
from transformers import get_cosine_schedule_with_warmup
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import math
from collections import OrderedDict
from torch_optimizer import Ranger
import cv2
from pathlib import Path
from tqdm.auto import tqdm
torch.multiprocessing.set_sharing_strategy('file_descriptor')

2024-07-11 12:22:05.994312: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-11 12:22:06.122861: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 12:22:06.122925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 12:22:06.144509: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-11 12:22:06.197521: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def make_mask(boxes):
    mask = np.zeros((IMG_SIZE[0], IMG_SIZE[1]))
    mask[...] = 1.0
    
    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = box
            mask[y1:y2+1, x1:x2+1] = 1.0
    return mask

def gen_seg_mask(mask):
    
    msk = np.zeros((mask.shape[0], mask.shape[1]))
    inv_msk = np.zeros((mask.shape[0], mask.shape[1]))
    for row in range(mask.shape[0]):
        for col in range(mask.shape[1]):
            if mask[row, col] == 127:
                msk[row, col] = 0
                inv_msk[row, col] = 1
            else:
                msk[row, col] = 1
                inv_msk[row, col] = 0
                
    return msk, inv_msk

In [ ]:
all_files = glob(f'cvt_png/*/*/*.png')
all_files_masks = glob(f'mask_png/*/*/*.png')

cdr = 'combo_png'
if not Path(cdr).exists():
    os.mkdir(cdr)

for i in tqdm(range(len(all_files))):
    fn_image = all_files[i]
    fn_mask = all_files_masks[i]
    
    img = Image.open(fn_image).convert('L')
    img = img.resize((320, 320))
    img = np.array(img)
    mask = Image.open(fn_mask).convert('L')
    mask = mask.resize((320, 320))
    mask = np.array(mask)
    
    msk, inv_msk = gen_seg_mask(mask)
    result = img.astype(np.float32) * inv_msk + mask.astype(np.float32) * msk
    
    if not Path(cdr + '/'+'/'.join(all_files[i].split('/')[1:2])).exists():
        os.mkdir(cdr + '/'+'/'.join(all_files[i].split('/')[1:2]))
      
    if not Path(cdr + '/'+'/'.join(all_files[i].split('/')[1:3])).exists():
        os.mkdir(cdr + '/'+'/'.join(all_files[i].split('/')[1:3]))
        
    cv2.imwrite(cdr + '/'+'/'.join(all_files[i].split('/')[1:]), result)

    
    

  0%|          | 0/147218 [00:00<?, ?it/s]